In [1]:
import numpy as np
import matplotlib.pyplot as plt

# 케라스 모델 구성
import tensorflow as tf 
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.initializers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend,activations
from keras.callbacks import EarlyStopping, ModelCheckpoint

# 모델 저장
from keras.models import load_model

# 멀티 GPU 사용
# from keras.utils import multi_gpu_model

Using TensorFlow backend.


# 설정 변수 정의

## 작업 Directory 설정

In [2]:
# 학습 데이터 파일명
train_list_file = "/src/data/crack/Cracks/resize/imgs_crack_train.npy"
# 라벨 데이터 파일명
label_list_file = "/src/data/crack/Cracks/resize/labels_crack_train.npy"
# 분석 모델 디렉토리명ㅣ
model_dir = "/src/hyebin/model/TEST/KERAS-TEST-1"

## 학습 데이터 설정

In [43]:
# 학습할 데이터의 개수 (batch_size)
training_count = 20  # 40

## 최적화 함수 설정
# 최적화 함수 학습률
learning_rate = 0.00001

## 학습 수행 설정
# 최대 학습 수행 횟수
epoch_count = 5  # 10000

# 모델 생성을 위한 연산 함수 생성

## 합성곱 연산 생성 함수
convolution 연산 수행<br>

in_x : 입력 데이터<br>
kernel_size(default=5)<br>
strides(default=1)<br>
activation(default='relu')<br>
padding(default='same')

In [4]:
def conv(in_x, filters, kernel_size=7, strides=1, activation='relu', padding='same'):
    x = Conv2D(filters, 
               (kernel_size, kernel_size),
               strides=strides,
               activation='relu',
               kernel_initializer='he_normal',
               padding='same') (in_x)
    return x

## 폴링 연산 생성 함수
maxpooling 연산 수행<br>

in_x : 입력 데이터<br>
kernel_size(default=(2,2))<br>
strides(default=2)

In [5]:
def maxpooling(in_x, kernel_size=(2, 2), strides=2):
    x = MaxPooling2D(kernel_size, strides=strides) (in_x)
    return x

## up-sampling, up-convolution 연산 생성 함수
decoding 구간에서 사용하는 convolution 및 concatenate 수행<br>

d_x : concat시 합칠 데이터<br>
u_x : 확장할 데이터<br>
filters : 필터 수<br>
kernel_size(default=2)<br>
strides(default=2)<br>
activation(default='relu')<br>
axis : concat 수행할 축 설정(default='None')

In [6]:
def upConv(d_x, u_x, filters, kernel_size=7, strides=2, activation = 'relu', axis = None):
    
    x = Conv2DTranspose(filters, kernel_size, strides=strides, padding='same') (u_x)
    
    if axis != None:
        x = concatenate([x, d_x], axis=axis)
    else:
        x = concatenate([x, d_x])
    
    return x

## 원본(data), 예측(pred), 라벨(label) 데이터를 화면에 표시하는 함수

In [8]:
def showCrack(data, pred, label):
    
    # pyplot의 현재 전체 이미지 크기를 백업 후 설정함
    tmp_size = plt.rcParams["figure.figsize"]
    plt.rcParams["figure.figsize"] = (15,7)  # 너비, 높이 (in)
    
    # 원본 데이터 출력
    plt.subplot(1,3,1)  # 한 번에 여러 그래프 출력
    plt.imshow(data.reshape([512,512]), cmap='gray')
    plt.axis('off')
    
    # 예측 데이터 출력
    plt.subplot(1,3,2)
    plt.imshow(pred.reshape([512,512]), cmap='gray')
    plt.axis('off')
    
    # 라벨 데이터 출력
    plt.subplot(1,3,3)
    plt.imshow(label.reshape([512,512]), cmap='gray')
    plt.axis('off')    
    
    plt.show()
    plt.close()
    
    # 백업된 pyplot 이미지 크기를 복원함
    plt.rcParams["figure.figsize"] = tmp_size

# 모델 생성
<변수 명명 규칙><br>
→ [과정코드][연산종류코드]_[층수]_[순번]<br>
<br>
과정코드 : d(down), b(bottle-neck), u(up)<br>
연산종류코드 : c(conv), p(pooling), u(upconv), x(copy and crop)<br>
층수는 U-Net의 가장 상위층을 1층으로 하여, 하위로 내려갈수록 높아짐<br>

In [39]:
def uNet():

    # 균열 이미지 입력 Tensor
    inputs = Input((512,512,1))
    x = Lambda(lambda x: x) (inputs)

    base_filter_cnt = 16

    # 압축과정(down) : U의 내려가는 부분
    dc_1_1 = conv(x, base_filter_cnt)
    dc_1_2 = conv(dc_1_1, base_filter_cnt)
    dp_1 = maxpooling(dc_1_2)
    print('dp_1 : {}'.format(dp_1))

    dc_2_1 = conv(dp_1, base_filter_cnt*2)
    dc_2_2 = conv(dc_2_1, base_filter_cnt*2)
    dp_2 = maxpooling(dc_2_2)
    print('dp_2 : {}'.format(dp_2))

    dc_3_1 = conv(dp_2, base_filter_cnt*4)
    dc_3_2 = conv(dc_3_1, base_filter_cnt*4)
    dp_3 = maxpooling(dc_3_2)
    print('dp_3 : {}'.format(dp_3))

    # 병목(bottle-neck) : 특징 압축, U의 바닥 부분
    bc_4_1 = conv(dp_3, base_filter_cnt*8)
    bc_4_2 = conv(bc_4_1, base_filter_cnt*8)
    print('\nbc_4_2 : {}'.format(bc_4_2))

    # 확장과정(up) : U의 올라가는 부분
    uu_3 = upConv(dc_3_2, bc_4_2, base_filter_cnt*4)
    uc_3_1 = conv(uu_3, base_filter_cnt*4)
    uc_3_2 = conv(uc_3_1, base_filter_cnt*4)
    print('\nuc_3_2 : {}'.format(uc_3_2))

    uu_2 = upConv(dc_2_2, uc_3_2, base_filter_cnt*2)
    uc_2_1 = conv(uu_2, base_filter_cnt*2)
    uc_2_2 = conv(uc_2_1, base_filter_cnt*2)
    print('uc_2_2 : {}'.format(uc_2_2))

    uu_1 = upConv(dc_1_1, uc_2_2, base_filter_cnt)
    uc_1_1 = conv(uu_1, base_filter_cnt)
    uc_1_2 = conv(uc_1_1, base_filter_cnt)
    print('uc_1_2 : {}'.format(uc_1_2))

    # 최종 출력
    pred = conv(uc_1_2, 1, kernel_size=1, activation='sigmoid')
    print('\npred : {}'.format(pred))
    
    model = Model(inputs=[inputs], outputs=[pred])
    model.summary()
    
    return model

# 손실 함수 정의

In [38]:
def customLoss(label, pred):

    diff = abs(label-pred)  # abs : 절대값 return 
    weight = 0.97

    # Step 함수를 이용한 가중치 설정
    loss = backend.mean((backend.sign(activations.relu(diff)) * weight + 1) * backend.square(diff))
    
    return loss

# 학습 수행

## 데이터 로딩
<데이터 구조><br>
train_list : 정상 이미지 + 균열 (이미지 개수,512, 512, 1)<br>
label_list : 균열 라벨 이미지 (이미지 개수, 512, 512, 1)

In [11]:
print("load train data: ", train_list_file)
train_list = np.load(train_list_file, allow_pickle=True)
print("load complete")

print("load train label data: ", label_list_file)
label_list = np.load(label_list_file, allow_pickle=True)
print("load complete")

s = np.arange(label_list.shape[0])
np.random.shuffle(s)

train_list = train_list[s]
label_list = label_list[s]

train_list = train_list.astype(np.float32)
label_list = label_list.astype(np.float32)

print(train_list.shape)
print(label_list.shape)

load train data:  /src/data/crack/Cracks/resize/imgs_crack_train.npy
load train label data:  /src/data/crack/Cracks/resize/labels_crack_train.npy
(2924, 512, 512, 1)
(2924, 512, 512, 1)


## 모델 로딩
모델 로드 및 Multi-GPU 사용 설정

In [40]:
# 모델 로드
model = uNet()

# 멀티 GPU 사용
# model = multi_gpu_model(model, gpus=2)

dp_1 : Tensor("max_pooling2d_19/MaxPool:0", shape=(None, 256, 256, 16), dtype=float32)
dp_2 : Tensor("max_pooling2d_20/MaxPool:0", shape=(None, 128, 128, 32), dtype=float32)
dp_3 : Tensor("max_pooling2d_21/MaxPool:0", shape=(None, 64, 64, 64), dtype=float32)

bc_4_2 : Tensor("conv2d_89/Relu:0", shape=(None, 64, 64, 128), dtype=float32)

uc_3_2 : Tensor("conv2d_91/Relu:0", shape=(None, 128, 128, 64), dtype=float32)
uc_2_2 : Tensor("conv2d_93/Relu:0", shape=(None, 256, 256, 32), dtype=float32)
uc_1_2 : Tensor("conv2d_95/Relu:0", shape=(None, 512, 512, 16), dtype=float32)

pred : Tensor("conv2d_96/Relu:0", shape=(None, 512, 512, 1), dtype=float32)
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 512, 512, 1)  0                                            
_________________________________

# loss, optimizer 설정

In [41]:
model.compile(optimizer=Adam(lr=learning_rate), loss=customLoss)

# 학습 수행

In [44]:
# 과적합 방지 설정값 및 checkpoint 생성 설정값
earlystopper = EarlyStopping(patience=5, verbose=2)
checkpointer = ModelCheckpoint(model_dir + '.ckpt', verbose=2, save_best_only=True)

# 학습
results = model.fit(train_list, label_list, 
                    epochs = epoch_count,
                    batch_size=training_count,
                    callbacks=[earlystopper, checkpointer])

Epoch 1/5
2924/2924 [==============================] - 112s 38ms/step - loss: 0.0409
Epoch 2/5
2924/2924 [==============================] - 105s 36ms/step - loss: 0.0348
Epoch 3/5
2924/2924 [==============================] - 109s 37ms/step - loss: 0.0325
Epoch 4/5
2924/2924 [==============================] - 107s 37ms/step - loss: 0.0311
Epoch 5/5
2924/2924 [==============================] - 106s 36ms/step - loss: 0.0300


# 모델 저장

In [45]:
print("save model: %s"%model_dir)

model.save(model_dir)

print("\ncomplete.")

save model: /src/hyebin/model/TEST/KERAS-TEST-1

complete.
